In [1]:
!pip install matplotlib ipywidgets numpy torch

In [1]:
import sys,os
sys.path.append("..")
print("current path:", os.getcwd())
# sys.path.insert(0, "/Users/ladislas/Desktop/motor_control_agent")
from glob import glob
import matplotlib.pyplot as plt
import ipywidgets as ipw
from IPython.display import Audio
import numpy as np
import pickle

from communicative_agent import CommunicativeAgent
from lib.dataset_wrapper import Dataset
from lib.notebooks import show_ema

print("current path:", os.getcwd())
# sys.path.insert(0, "/Users/ladislas/Desktop/motor_control_agent")
sys.path.insert(0, "/mnt/c/Users/vpaul/Documents/Inner_Speech/agent/")
from external import lpcynet
#from external import *

current path: /mnt/c/Users/vpaul/Documents/Inner_Speech/agent/communicative_agent
current path: /mnt/c/Users/vpaul/Documents/Inner_Speech/agent/communicative_agent


In [2]:

print("current path:", os.getcwd())
# sys.path.insert(0, "/Users/ladislas/Desktop/motor_control_agent")


current path: /mnt/c/Users/vpaul/Documents/Inner_Speech/agent/communicative_agent


In [3]:
agents_path = glob("../out/communicative_agent/*/")
agents_path.sort()
print(agents_path)

['../out/communicative_agent/0458b50b843eb9a9cdd308c21f0c023f-4/', '../out/communicative_agent/0829fe5eb136a69bf08f1f7969363bc8-1/', '../out/communicative_agent/1317532187c0c0214be21256627b7033-4/', '../out/communicative_agent/255f581101c5169ce56d919198f62c33-2/', '../out/communicative_agent/25c7a23c9dbdb22b04d89cd29539d42b-4/', '../out/communicative_agent/29ad219b4b0a7b1f1a04feaea94149ed-0/', '../out/communicative_agent/2ab5a89f4f49a9598d6b004ef67ff3e1-4/', '../out/communicative_agent/2d2c8282327ffc71177907b6649a441e-1/', '../out/communicative_agent/2dc4661008775a91132083da4874e49e-1/', '../out/communicative_agent/4936c9a76095bd94ddd8ce4f066b8bde-2/', '../out/communicative_agent/4ea8e7d887c3aa4692d5c1a162087f11-1/', '../out/communicative_agent/5712219048d80e333c923cb044ec1e95-2/', '../out/communicative_agent/5b6a02d2fc806a8d8f6c93b3d0fb6202-4/', '../out/communicative_agent/5e5dce5ba23ccc653b8cb0cbd29c7cc9-0/', '../out/communicative_agent/6c31ae73341624b9c8dcc87fe0785d2a-0/', '../out/c

In [4]:
agents_alias = []

for agent_path in agents_path:
    agent = CommunicativeAgent.reload(agent_path, load_nn=False)
    config = agent.config
    
    if agent.synthesizer.config['dataset']['art_type'] == "ema":
        continue
    
    agent_alias = " ".join((
        agent_path[-5:-1],
        f"{','.join(agent.sound_quantizer.config['dataset']['names'])}",
        f"synth_art={agent.synthesizer.config['dataset']['art_type']}",
        f"jerk={config['training']['jerk_loss_weight']}",
    ))
    
    agents_alias.append((agent_alias, agent_path))

KeyboardInterrupt: 

In [18]:
!pip install notebook ipywidgets
!jupyter nbextension enable --py widgetsnbextension


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 4.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.3/380.3 KB 2.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 2.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.4/256.4 KB 2.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 KB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 KB 1.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 2.3 MB/s eta 0

In [22]:
!jupyter-nbextension list #enable --py widgetsnbextension --sys-prefix



/bin/bash: line 1: jupyter-nbextension: command not found


In [8]:
def show_agent(agent_path):
    try:
        agent = CommunicativeAgent.reload(agent_path)
        
        sound_type = agent.synthesizer.config["dataset"]["sound_type"]
        art_type = agent.synthesizer.config["dataset"]["art_type"]
        synth_dataset = agent.synthesizer.dataset
        
        def show_dataset(dataset_name):
            try :
                dataset = Dataset(dataset_name)
                items_cepstrum = dataset.get_items_data(sound_type, cut_silences=True)
                items_source = dataset.get_items_data("source", cut_silences=True)
                sampling_rate = dataset.features_config["wav_sampling_rate"]
                
                # items_ema = dataset.get_items_data("ema", cut_silences=True)
                
                items_name = dataset.get_items_list()
                
                def resynth_item(item_name):
                    try :
                        item_cepstrum = items_cepstrum[item_name]
                        item_source = items_source[item_name]
                        item_wave = dataset.get_item_wave(item_name)
                        nb_frames = len(item_cepstrum)
                        
                        repetition = agent.repeat(item_cepstrum)
                        repeated_cepstrum = repetition["sound_repeated"]
                        estimated_cepstrum = repetition["sound_estimated"]
                        estimated_art = repetition["art_estimated"]
                        
                        repeated_sound = np.concatenate((repeated_cepstrum, item_source), axis=1)
                        estimated_sound = np.concatenate((estimated_cepstrum, item_source), axis=1)

                        repeated_wave = lpcynet.synthesize_frames(repeated_sound)
                        estimated_wave = lpcynet.synthesize_frames(estimated_sound)
                        
                        print("Original sound:")
                        display(Audio(item_wave, rate=sampling_rate))
                        print("Repetition (Inverse model → Synthesizer → LPCNet):")
                        display(Audio(repeated_wave, rate=sampling_rate))
                        print("Estimation (Inverse model → Direct model → LPCNet):")
                        display(Audio(estimated_wave, rate=sampling_rate))
                        
                        plt.figure(figsize=(nb_frames/20, 6), dpi=120)
                        
                        ax = plt.subplot(311)
                        ax.set_title("original %s" % (sound_type))
                        ax.imshow(item_cepstrum.T, origin="lower")
                        
                        ax = plt.subplot(312)
                        ax.set_title("Repetition")
                        ax.imshow(repeated_cepstrum.T, origin="lower")
                        
                        ax = plt.subplot(313)
                        ax.set_title("Estimation")
                        ax.imshow(estimated_cepstrum.T, origin="lower")
                        
                        plt.tight_layout()
                        plt.show()
                        
                        if art_type == "art_params":
                            estimated_art = synth_dataset.art_to_ema(estimated_art)
                        # item_ema = items_ema[item_name]
                        show_ema(estimated_art, reference=None, dataset=synth_dataset)
                    except Exception as e:
                        logger.error(f"Error in resynth_item: {e}", exc_info=True)
                
                display(ipw.interactive(resynth_item, item_name=items_name))
            except Exception as e:
                    logger.error(f"Error in show_dataset: {e}", exc_info=True)
        display(ipw.interactive(show_dataset, dataset_name=agent.sound_quantizer.config["dataset"]["names"]))
    except Exception as e:
        logger.error(f"Error in show_agent: {e}", exc_info=True)

# New function to handle agent selection and display
# def select_agent(agents_alias):
#     def on_agent_selected(change):
#         if change['type'] == 'change' and change['name'] == 'value':
#             # When an agent is selected from the dropdown, show_agent is called with the selected path
#             show_agent(change['new'][1])  # change['new'] is the selected tuple, [1] is the path

#     # Creating a dropdown widget for agent selection
#     agent_dropdown = ipw.Dropdown(options=agents_alias, description="Select Agent:")
#     agent_dropdown.observe(on_agent_selected, names='value')

#     # Displaying the dropdown
#     display(agent_dropdown)

# # Call the function to display the dropdown with agents_alias as the options
# select_agent(agents_alias)

import ipywidgets as ipw

# Existing functions: show_agent, show_dataset, resynth_item

# Assume agents_alias and show_agent function are defined as before

# Function to handle selection from the dropdown
# def on_agent_selected(change):
#     if change['type'] == 'change' and change['name'] == 'value':
#         agent_path = change['new']
#         show_agent(agent_path)

# # Creating a dropdown widget for agent selection
# agent_dropdown = ipw.Dropdown(
#     options=[(alias, path) for alias, path in agents_alias],
#     description="Select Agent:"
# )

# Observe changes in the dropdown value
#agent_dropdown.observe(on_agent_selected, names='value')

# Display the dropdown menu
#display(agent_dropdown)

#print(agents_alias)
display(ipw.interactive(show_agent, agent_path=agents_alias))

interactive(children=(Dropdown(description='agent_path', options=(('3f-4 pb2007 synth_art=art_params jerk=0.1'…

In [6]:
import ipywidgets as ipw

# Define a simple function
def multiply(a, b):
    print(f"{a} * {b} = {a * b}")

# Use interactive to create UI controls for the function
interactive_widget = ipw.interactive(multiply, a=10, b=5)

# Display the interactive widgets
interactive_widget

interactive(children=(IntSlider(value=10, description='a', max=30, min=-10), IntSlider(value=5, description='b…

In [1]:
display(ipw.interactive(show_agent, agent_path=agents_alias))

NameError: name 'ipw' is not defined

In [8]:
from external.lpcynet import *
def resynth_item(item_name):
    item_cepstrum = items_cepstrum[item_name]
    item_source = items_source[item_name]
    item_wave = dataset.get_item_wave(item_name)
    nb_frames = len(item_cepstrum)

    repetition = agent.repeat(item_cepstrum)
    repeated_cepstrum = repetition["sound_repeated"]
    estimated_cepstrum = repetition["sound_estimated"]
    estimated_art = repetition["art_estimated"]

    repeated_sound = np.concatenate((repeated_cepstrum, item_source), axis=1)
    estimated_sound = np.concatenate((estimated_cepstrum, item_source), axis=1)

    repeated_wave = lpcynet.synthesize_frames(repeated_sound)
    estimated_wave = lpcynet.synthesize_frames(estimated_sound)

    print("Original sound:")
    display(Audio(item_wave, rate=sampling_rate))
    print("Repetition (Inverse model → Synthesizer → LPCNet):")
    display(Audio(repeated_wave, rate=sampling_rate))
    print("Estimation (Inverse model → Direct model → LPCNet):")
    display(Audio(estimated_wave, rate=sampling_rate))

    # Plotting the cepstrums
    plt.figure(figsize=(15, 9))

    # Plot original cepstrum
    ax1 = plt.subplot(311)
    ax1.imshow(item_cepstrum.T, origin="lower", aspect='auto')
    ax1.set_title("Original Cepstrum")
    ax1.set_ylabel("Cepstral Coefficients")
    ax1.set_xlabel("Frames")

    # Plot repeated cepstrum
    ax2 = plt.subplot(312, sharex=ax1, sharey=ax1)
    ax2.imshow(repeated_cepstrum.T, origin="lower", aspect='auto')
    ax2.set_title("Repeated Cepstrum")
    ax2.set_ylabel("Cepstral Coefficients")
    ax2.set_xlabel("Frames")

    # Plot estimated cepstrum
    ax3 = plt.subplot(313, sharex=ax1, sharey=ax1)
    ax3.imshow(estimated_cepstrum.T, origin="lower", aspect='auto')
    ax3.set_title("Estimated Cepstrum")
    ax3.set_ylabel("Cepstral Coefficients")
    ax3.set_xlabel("Frames")

    plt.tight_layout()
    plt.show()

# Manually specify the agent path and dataset name
specific_agent_path = agents_path[1]  # Replace with the actual path
specific_dataset_name = "pb2007"    # Replace with the actual dataset name
specific_item_name = dataset.get_items_list()
           # Replace with an actual item name from the dataset

# Load the agent and dataset
agent = CommunicativeAgent.reload(specific_agent_path)
dataset = Dataset(specific_dataset_name)
items_cepstrum = dataset.get_items_data(agent.synthesizer.config["dataset"]["sound_type"], cut_silences=True)
items_source = dataset.get_items_data("source", cut_silences=True)

# Call the resynth_item function directly
resynth_item(specific_item_name)

NameError: name 'dataset' is not defined

In [ ]:
os.getcwd()

os.chdir(r"c:\\Users\\vpaul\\Documents\\Inner_Speech\\agent\\communicative_agent")

In [ ]:
sys.path.insert(0, "C:/Users/vpaul/Documents/Inner_Speech/agent/external/")

!pip install lpcynet

ERROR: Could not find a version that satisfies the requirement lpcynet (from versions: none)
ERROR: No matching distribution found for lpcynet


In [11]:

import sys,os

# Clear the current sys.path
sys.path.clear()

# Add paths as per the terminal's sys.path
sys.path.extend([
    'C:\\Users\\vpaul\\anaconda3\\python311.zip',
    'C:\\Users\\vpaul\\anaconda3\\Lib',
    'C:\\Users\\vpaul\\anaconda3\\DLLs',
    'C:\\Users\\vpaul\\anaconda3',
    'C:\\Users\\vpaul\\anaconda3\\Lib\\site-packages',
    'C:\\Users\\vpaul\\anaconda3\\Lib\\site-packages\\win32',
    'C:\\Users\\vpaul\\anaconda3\\Lib\\site-packages\\win32\\lib',
    'C:\\Users\\vpaul\\anaconda3\\Lib\\site-packages\\Pythonwin'
])

print("current path:", os.getcwd())
# sys.path.insert(0, "/Users/ladislas/Desktop/motor_control_agent")
#sys.path.insert( 0,"C:/Users/vpaul/Documents/Inner_Speech/agent")
from external import lpcynet
print(lpcynet.__file__)
print(hasattr(lpcynet, 'synthesize_frames'))

current path: c:\Users\vpaul\Documents\Inner_Speech\agent\communicative_agent
C:\Users/vpaul/Documents/Inner_Speech/agent\external\lpcynet\__init__.py
False


In [35]:
from external.lpcynet import *

In [27]:

print(dir(lpcynet))
print(lpcynet.__package__)
print(lpcynet.__builtins__)
print(lpcynet.__cached__)
print(lpcynet.__doc__)
print(lpcynet.__loader__)
print(lpcynet.__name__)
print(lpcynet.__path__)
print(lpcynet.__spec__)


['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__']
external.lpcynet
{'__name__': 'builtins', '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.", '__package__': '', '__loader__': <class '_frozen_importlib.BuiltinImporter'>, '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>, origin='built-in'), '__build_class__': <built-in function __build_class__>, '__import__': <built-in function __import__>, 'abs': <built-in function abs>, 'all': <built-in function all>, 'any': <built-in function any>, 'ascii': <built-in function ascii>, 'bin': <built-in function bin>, 'breakpoint': <built-in function breakpoint>, 'callable': <built-in function callable>, 'chr': <built-in function chr>, 'compile': <built-in function compile>, 'delattr': <built-in function delattr>, 'dir': <built-in function dir>, 'divmod'

In [9]:
# Assuming agents_path, CommunicativeAgent, and Dataset classes are defined as per your previous code
import sys,os
sys.path.append( "/mnt/c/Users/vpaul/Documents/Inner_Speech/agent")
from external.lpcynet import lpcynet
print(lpcynet.__file__)
print(hasattr(lpcynet, 'analyze_frames'))

# Select the first agent path

specific_agent_path = agents_path[0] if agents_path else None


    # Define resynth_item function here
def resynth_item(item_name, items_cepstrum, items_source, agent, dataset):
    item_cepstrum = items_cepstrum[item_name]
    item_source = items_source[item_name]
    item_wave = dataset.get_item_wave(item_name)
    nb_frames = len(item_cepstrum)

    repetition = agent.repeat(item_cepstrum)
    repeated_cepstrum = repetition["sound_repeated"]
    estimated_cepstrum = repetition["sound_estimated"]
    estimated_art = repetition["art_estimated"]

    repeated_sound = np.concatenate((repeated_cepstrum, item_source), axis=1)
    estimated_sound = np.concatenate((estimated_cepstrum, item_source), axis=1)

    repeated_wave = lpcynet.analyze_frames(repeated_sound)
    estimated_wave = lpcynet.synthesize_frames(estimated_sound)

    print("Original sound:")
    display(Audio(item_wave, rate=sampling_rate))
    print("Repetition (Inverse model → Synthesizer → LPCNet):")
    display(Audio(repeated_wave, rate=sampling_rate))
    print("Estimation (Inverse model → Direct model → LPCNet):")
    display(Audio(estimated_wave, rate=sampling_rate))

    # Plotting the cepstrums
    plt.figure(figsize=(15, 9))

    # Plot original cepstrum
    ax1 = plt.subplot(311)
    ax1.imshow(item_cepstrum.T, origin="lower", aspect='auto')
    ax1.set_title("Original Cepstrum")
    ax1.set_ylabel("Cepstral Coefficients")
    ax1.set_xlabel("Frames")

    # Plot repeated cepstrum
    ax2 = plt.subplot(312, sharex=ax1, sharey=ax1)
    ax2.imshow(repeated_cepstrum.T, origin="lower", aspect='auto')
    ax2.set_title("Repeated Cepstrum")
    ax2.set_ylabel("Cepstral Coefficients")
    ax2.set_xlabel("Frames")

    # Plot estimated cepstrum
    ax3 = plt.subplot(313, sharex=ax1, sharey=ax1)
    ax3.imshow(estimated_cepstrum.T, origin="lower", aspect='auto')
    ax3.set_title("Estimated Cepstrum")
    ax3.set_ylabel("Cepstral Coefficients")
    ax3.set_xlabel("Frames")

    plt.tight_layout()
    plt.show()
# Load the agent
if specific_agent_path:
    agent = CommunicativeAgent.reload(specific_agent_path, load_nn=True)
    specific_dataset_name = agent.sound_quantizer.config['dataset']['names'][0]  # First dataset name

    # Load the dataset
    dataset = Dataset(specific_dataset_name)
    items_cepstrum = dataset.get_items_data(agent.synthesizer.config["dataset"]["sound_type"], cut_silences=True)
    items_source = dataset.get_items_data("source", cut_silences=True)

    # Get an example item name
    items_name = dataset.get_items_list()
    specific_item_name = items_name[0] if items_name else None  # First item name

    if specific_item_name:
        # Ensure specific_item_name is a string, not a tuple
        # Check if the item name exists in items_cepstrum
        if specific_item_name in items_cepstrum:
            resynth_item(specific_item_name, items_cepstrum, items_source, agent, dataset)
        else:
            print(f"Item name '{specific_item_name}' not found in items_cepstrum.")
            print("Available item names:", list(items_cepstrum.keys()))
        if isinstance(specific_item_name, tuple):
            specific_item_name = specific_item_name[1]

        # Call the resynth_item function
        resynth_item(specific_item_name, items_cepstrum, items_source, agent, dataset)
    else:
        print("No items found in the dataset.")
else:
    print("No agent paths found.")


/mnt/c/Users/vpaul/Documents/Inner_Speech/agent/external/lpcynet/lpcynet.cpython-310-x86_64-linux-gnu.so
True
Item name '('item_0000 a', 'item_0000')' not found in items_cepstrum.
Available item names: ['item_0000', 'item_0001', 'item_0002', 'item_0003', 'item_0004', 'item_0005', 'item_0006', 'item_0007', 'item_0008', 'item_0009', 'item_0010', 'item_0011', 'item_0012', 'item_0013', 'item_0014', 'item_0015', 'item_0016', 'item_0017', 'item_0018', 'item_0019', 'item_0020', 'item_0021', 'item_0022', 'item_0023', 'item_0024', 'item_0025', 'item_0026', 'item_0027', 'item_0028', 'item_0029', 'item_0030', 'item_0031', 'item_0032', 'item_0033', 'item_0034', 'item_0035', 'item_0036', 'item_0037', 'item_0038', 'item_0039', 'item_0040', 'item_0041', 'item_0042', 'item_0043', 'item_0044', 'item_0045', 'item_0046', 'item_0047', 'item_0048', 'item_0049', 'item_0050', 'item_0051', 'item_0052', 'item_0053', 'item_0054', 'item_0055', 'item_0056', 'item_0057', 'item_0058', 'item_0059', 'item_0060', 'ite

ValueError: could not broadcast input array from shape (2,) into shape (0,)

In [10]:
def show_agent(agent_path):
    agent = CommunicativeAgent.reload(agent_path)
    
    sound_type = agent.synthesizer.config["dataset"]["sound_type"]
    art_type = agent.synthesizer.config["dataset"]["art_type"]
    synth_dataset = agent.synthesizer.dataset
    
    def show_dataset(dataset_name):
        dataset = Dataset(dataset_name)
        items_cepstrum = dataset.get_items_data(sound_type, cut_silences=True)
        items_source = dataset.get_items_data("source", cut_silences=True)
        sampling_rate = dataset.features_config["wav_sampling_rate"]
        
        # items_ema = dataset.get_items_data("ema", cut_silences=True)
        
        items_name = dataset.get_items_list()
        
        def resynth_item(item_name):
            item_cepstrum = items_cepstrum[item_name]
            item_source = items_source[item_name]
            item_wave = dataset.get_item_wave(item_name)
            nb_frames = len(item_cepstrum)

            repetition = agent.repeat(item_cepstrum)
            repeated_cepstrum = repetition["sound_repeated"]
            estimated_cepstrum = repetition["sound_estimated"]
            estimated_art = repetition["art_estimated"]

            repeated_sound = np.concatenate((repeated_cepstrum, item_source), axis=1)
            estimated_sound = np.concatenate((estimated_cepstrum, item_source), axis=1)

            repeated_wave = lpcynet.synthesize_frames(repeated_sound)
            estimated_wave = lpcynet.synthesize_frames(estimated_sound)

            print("Original sound:")
            display(Audio(item_wave, rate=sampling_rate))
            print("Repetition (Inverse model → Synthesizer → LPCNet):")
            display(Audio(repeated_wave, rate=sampling_rate))
            print("Estimation (Inverse model → Direct model → LPCNet):")
            display(Audio(estimated_wave, rate=sampling_rate))

            # Plotting the cepstrums
            plt.figure(figsize=(15, 9))

            # Plot original cepstrum
            ax1 = plt.subplot(311)
            ax1.imshow(item_cepstrum.T, origin="lower", aspect='auto')
            ax1.set_title("Original Cepstrum")
            ax1.set_ylabel("Cepstral Coefficients")
            ax1.set_xlabel("Frames")

            # Plot repeated cepstrum
            ax2 = plt.subplot(312, sharex=ax1, sharey=ax1)
            ax2.imshow(repeated_cepstrum.T, origin="lower", aspect='auto')
            ax2.set_title("Repeated Cepstrum")
            ax2.set_ylabel("Cepstral Coefficients")
            ax2.set_xlabel("Frames")

            # Plot estimated cepstrum
            ax3 = plt.subplot(313, sharex=ax1, sharey=ax1)
            ax3.imshow(estimated_cepstrum.T, origin="lower", aspect='auto')
            ax3.set_title("Estimated Cepstrum")
            ax3.set_ylabel("Cepstral Coefficients")
            ax3.set_xlabel("Frames")

            plt.tight_layout()
            plt.show()
            
            if art_type == "art_params":
                estimated_art = synth_dataset.art_to_ema(estimated_art)
            # item_ema = items_ema[item_name]
            show_ema(estimated_art, reference=None, dataset=synth_dataset)

        
        display(ipw.interactive(resynth_item, item_name=items_name))
    display(ipw.interactive(show_dataset, dataset_name=agent.sound_quantizer.config["dataset"]["names"]))

import ipywidgets as ipw

import ipywidgets as ipw

# Assume agents_alias and show_agent function are defined as before

# Function to handle selection from the dropdown
def on_agent_selected(change):
    if change['type'] == 'change' and change['name'] == 'value':
        agent_path = change['new']
        show_agent(agent_path)

# Creating a dropdown widget for agent selection
agent_dropdown = ipw.Dropdown(
    options=[(alias, path) for alias, path in agents_alias],
    description="Select Agent:"
)

# Observe changes in the dropdown value
agent_dropdown.observe(on_agent_selected, names='value')

# Display the dropdown menu
display(agent_dropdown)

Dropdown(description='Select Agent:', options=(('3f-4 pb2007 synth_art=art_params jerk=0.1', '../out/communica…

interactive(children=(Dropdown(description='dataset_name', options=('pb2007',), value='pb2007'), Output()), _d…

interactive(children=(Dropdown(description='dataset_name', options=('pb2007',), value='pb2007'), Output()), _d…

In [11]:
import ipywidgets as ipw
from IPython.display import display, Audio
import matplotlib.pyplot as plt
import numpy as np

def show_agent(agent_path):
    try:
        agent = CommunicativeAgent.reload(agent_path)
        sound_type = agent.synthesizer.config["dataset"]["sound_type"]
        art_type = agent.synthesizer.config["dataset"]["art_type"]
        synth_dataset = agent.synthesizer.dataset

        def show_dataset(dataset_name):
            dataset = Dataset(dataset_name)
            items_cepstrum = dataset.get_items_data(sound_type, cut_silences=True)
            items_source = dataset.get_items_data("source", cut_silences=True)
            sampling_rate = dataset.features_config["wav_sampling_rate"]
            items_name = dataset.get_items_list()

            # Define resynth_item as before

            # Interactive widget for resynth_item
            item_selector = ipw.interactive(resynth_item, item_name=items_name)
            display(item_selector)

        # Manually select the first dataset to display
        if agent.sound_quantizer.config["dataset"]["names"]:
            first_dataset_name = agent.sound_quantizer.config["dataset"]["names"][0]
            show_dataset(first_dataset_name)
        else:
            print("No datasets found.")

    except Exception as e:
        print(f"An error occurred: {e}")

# Dropdown for agent selection
agent_dropdown = ipw.Dropdown(
    options=[(alias, path) for alias, path in agents_alias],
    description="Select Agent:"
)

# Function to handle selection from the dropdown
def on_agent_selected(change):
    if change['type'] == 'change' and change['name'] == 'value':
        show_agent(change['new'])

# Observe changes in the dropdown value
agent_dropdown.observe(on_agent_selected, names='value')

# Display the dropdown menu
display(agent_dropdown)


Dropdown(description='Select Agent:', options=(('3f-4 pb2007 synth_art=art_params jerk=0.1', '../out/communica…

An error occurred: cannot find widget or abbreviation for argument: 'items_cepstrum'
An error occurred: cannot find widget or abbreviation for argument: 'items_cepstrum'
An error occurred: cannot find widget or abbreviation for argument: 'items_cepstrum'
An error occurred: cannot find widget or abbreviation for argument: 'items_cepstrum'
